In [2]:
import pandas as pd
import numpy as np
import re
import time
import swifter
source_data = pd.read_csv("hardware_zone_raw_data_with_detail_multithread.csv")

In [3]:
print(source_data.shape)
source_data['Timestamp'] = pd.to_datetime(source_data['Timestamp'])
source_data = source_data[(source_data['Timestamp'] > '2017-01-01 00:00:00')]
source_data['Search_Term'] = source_data['Search_Term'].astype(str)
source_data['Content'] =  source_data['Content'].astype(str)
print(source_data.shape)


(1140, 12)
(1065, 12)
(1140, 12)
(1065, 12)


In [4]:
quote_string = "Click to expand..."
def content_cleaning(row):
    final_string = ""
    quote_index = str(row.Content).find(quote_string)
    if quote_index > 0:
        final_string = row.Content[quote_index + len(quote_string):]
    else:
        final_string = row.Content
    
    #Weird thing have to do this to convert to unicode then removed by ascii
    pattern = r"\\u.{4}"
    mod_string = re.sub(pattern, ' ', final_string)

    #encode() method
    strencode = str(mod_string).encode("ascii", "ignore")
    #decode() method
    strdecode = strencode.decode()

    return strdecode
print("Start cleaning at " + time.ctime())
source_data.loc[:,"Content"] = source_data.swifter.apply(content_cleaning, axis=1)

Start cleaning at Tue Apr  4 13:38:40 2023
Start cleaning at Tue Apr  4 13:38:40 2023


Pandas Apply:   0%|          | 0/1065 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1065 [00:00<?, ?it/s]

In [5]:

print("Start Case Fold at " + time.ctime())
source_data.loc[:,"Search_Term"] = source_data["Search_Term"].str.lower()
source_data.loc[:,"Content"] = source_data["Content"].str.lower()

print("End Case Fold at " + time.ctime())
print(source_data.shape)

Start Case Fold at Tue Apr  4 13:38:43 2023
End Case Fold at Tue Apr  4 13:38:43 2023
(1065, 12)
Start Case Fold at Tue Apr  4 13:38:43 2023
End Case Fold at Tue Apr  4 13:38:43 2023
(1065, 12)


In [6]:
# Find exact match no split

def find_exact_match_no_split(row):
    search_token = row.Search_Term
    matched = True
    search_token = r'\b' + re.escape(search_token) + r'\b'
    found = re.search(search_token, row.Content)
    if not found:
        matched = False
            
    return matched



In [7]:
print("Finding Matching Word Start at " + time.ctime())

source_data.loc[:,"Matching_Word"] = source_data.swifter.apply(find_exact_match_no_split, axis=1)
print("Finding Matching Word End at " + time.ctime())

Finding Matching Word Start at Tue Apr  4 13:38:44 2023
Finding Matching Word Start at Tue Apr  4 13:38:44 2023


Pandas Apply:   0%|          | 0/1065 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1065 [00:00<?, ?it/s]

Finding Matching Word End at Tue Apr  4 13:38:44 2023
Finding Matching Word End at Tue Apr  4 13:38:44 2023


In [8]:
source_data = source_data[source_data['Matching_Word'] == True]
source_data.head()

,Unnamed: 0,User_id,Type,Content_id,Url,Reply_to_content_id,Thread_id,Subforum,Title,Content,Timestamp,Search_Term,Matching_Word
1,1,204542,Post,['146188473'],https://forums.hardwarezone.com.sg/threads/hou...,NaN,6870671.0,Forums,"Household incomes rose in 2022, income inequal...",singapore - median household income grew in 20...,2023-02-09 10:21:12,income inequality,True
2,2,474880,Post,['115313181'],https://forums.hardwarezone.com.sg/threads/bac...,NaN,NaN,Forums,NaN,"hi, want to get views from you all. what does ...",2018-07-05 15:55:32,upper class,True
10,10,664200,Post,['113150442'],https://forums.hardwarezone.com.sg/threads/cpf...,NaN,5787403.0,Forums,CPF questions : Medisave after 55 & RA Topup,1. medisave after 55 as i understand excess in...,2018-03-06 23:26:17,medisave,True
16,16,815074,Post,['145992640'],https://forums.hardwarezone.com.sg/threads/can...,NaN,6866152.0,Forums,"Can CDAC directors OYK, Sun XL, Koh PK, Chee H...",cdac audited report fy 2021 read cdac audited ...,2023-01-27 00:43:39,cdac,True
24,24,789529,Post,['146795555'],https://forums.hardwarezone.com.sg/threads/new...,NaN,NaN,Forums,NaN,accident victim can claim govt medical subsidi...,2023-03-22 11:59:32,medical expenses,True


,Unnamed: 0,User_id,Type,Content_id,Url,Reply_to_content_id,Thread_id,Subforum,Title,Content,Timestamp,Search_Term,Matching_Word
1,1,204542,Post,['146188473'],https://forums.hardwarezone.com.sg/threads/hou...,NaN,6870671.0,Forums,"Household incomes rose in 2022, income inequal...",singapore - median household income grew in 20...,2023-02-09 10:21:12,income inequality,True
2,2,474880,Post,['115313181'],https://forums.hardwarezone.com.sg/threads/bac...,NaN,NaN,Forums,NaN,"hi, want to get views from you all. what does ...",2018-07-05 15:55:32,upper class,True
10,10,664200,Post,['113150442'],https://forums.hardwarezone.com.sg/threads/cpf...,NaN,5787403.0,Forums,CPF questions : Medisave after 55 & RA Topup,1. medisave after 55 as i understand excess in...,2018-03-06 23:26:17,medisave,True
16,16,815074,Post,['145992640'],https://forums.hardwarezone.com.sg/threads/can...,NaN,6866152.0,Forums,"Can CDAC directors OYK, Sun XL, Koh PK, Chee H...",cdac audited report fy 2021 read cdac audited ...,2023-01-27 00:43:39,cdac,True
24,24,789529,Post,['146795555'],https://forums.hardwarezone.com.sg/threads/new...,NaN,NaN,Forums,NaN,accident victim can claim govt medical subsidi...,2023-03-22 11:59:32,medical expenses,True


In [9]:
print(source_data.shape)


(110, 13)
(110, 13)


In [10]:
keywords = ['middle income', 'high income', 'low income', 'income inequality', 'disposable income', 'household annual income',
            'wealth tax', 'unemployment insurance', 'meritocracy', 'occupation mobility', 'elite education', 'education subsidies', 'Mendaki',
            'CDAC', 'less educated', 'health insurance', 'medical expenses', 'medical resources', 'upper class', 'lower class',
           'middle class', 'central provident fund', 'CPF']

keywords = set([x.lower() for x in keywords])


In [11]:
print(keywords)

def hasOtherSentiment(row, keyword):
    if row.Search_Term != keyword and keyword in row.Content:
        return True
    else:
        return False







{'meritocracy', 'medical expenses', 'income inequality', 'cpf', 'middle income', 'wealth tax', 'mendaki', 'elite education', 'high income', 'middle class', 'health insurance', 'low income', 'upper class', 'less educated', 'education subsidies', 'disposable income', 'occupation mobility', 'central provident fund', 'medical resources', 'household annual income', 'cdac', 'unemployment insurance', 'lower class'}
{'meritocracy', 'medical expenses', 'income inequality', 'cpf', 'middle income', 'wealth tax', 'mendaki', 'elite education', 'high income', 'middle class', 'health insurance', 'low income', 'upper class', 'less educated', 'education subsidies', 'disposable income', 'occupation mobility', 'central provident fund', 'medical resources', 'household annual income', 'cdac', 'unemployment insurance', 'lower class'}


In [12]:
frames = []
frames.append(source_data)
backup = source_data

for keyword in keywords:
    source_date = backup
    source_data.loc[:,"Has_Other_Sentiment"] = source_data.swifter.apply(hasOtherSentiment, keyword=keyword, axis=1)
    other_df = source_data[source_data['Has_Other_Sentiment'] == True]
    if not other_df.empty:
        other_df.loc[:,"Search_Term"] = keyword
        other_df.head()
        frames.append(other_df)
        print("Found additional search term " + keyword)
    else:
        print("No additional search term " + keyword)


Pandas Apply:   0%|          | 0/110 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/110 [00:00<?, ?it/s]

No additional search term meritocracy
No additional search term meritocracy


C:\Users\Chuan Wu\AppData\Local\Temp\ipykernel_64928\1100374070.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_data.loc[:,"Has_Other_Sentiment"] = source_data.swifter.apply(hasOtherSentiment, keyword=keyword, axis=1)
C:\Users\Chuan Wu\AppData\Local\Temp\ipykernel_64928\1100374070.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_data.loc[:,"Has_Other_Sentiment"] = source_data.swifter.apply(hasOtherSentiment, keyword=keyword, axis=1)


Pandas Apply:   0%|          | 0/110 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/110 [00:00<?, ?it/s]

No additional search term unemployment insurance
No additional search term unemployment insurance


Pandas Apply:   0%|          | 0/110 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/110 [00:00<?, ?it/s]

No additional search term lower class
No additional search term lower class


In [13]:
result = pd.concat(frames)
print(result.shape)
result.head()

(139, 14)


,Unnamed: 0,User_id,Type,Content_id,Url,Reply_to_content_id,Thread_id,Subforum,Title,Content,Timestamp,Search_Term,Matching_Word,Has_Other_Sentiment
1,1,204542,Post,['146188473'],https://forums.hardwarezone.com.sg/threads/hou...,NaN,6870671.0,Forums,"Household incomes rose in 2022, income inequal...",singapore - median household income grew in 20...,2023-02-09 10:21:12,income inequality,True,False
2,2,474880,Post,['115313181'],https://forums.hardwarezone.com.sg/threads/bac...,NaN,NaN,Forums,NaN,"hi, want to get views from you all. what does ...",2018-07-05 15:55:32,upper class,True,False
10,10,664200,Post,['113150442'],https://forums.hardwarezone.com.sg/threads/cpf...,NaN,5787403.0,Forums,CPF questions : Medisave after 55 & RA Topup,1. medisave after 55 as i understand excess in...,2018-03-06 23:26:17,medisave,True,False
16,16,815074,Post,['145992640'],https://forums.hardwarezone.com.sg/threads/can...,NaN,6866152.0,Forums,"Can CDAC directors OYK, Sun XL, Koh PK, Chee H...",cdac audited report fy 2021 read cdac audited ...,2023-01-27 00:43:39,cdac,True,False
24,24,789529,Post,['146795555'],https://forums.hardwarezone.com.sg/threads/new...,NaN,NaN,Forums,NaN,accident victim can claim govt medical subsidi...,2023-03-22 11:59:32,medical expenses,True,False


(139, 14)


,Unnamed: 0,User_id,Type,Content_id,Url,Reply_to_content_id,Thread_id,Subforum,Title,Content,Timestamp,Search_Term,Matching_Word,Has_Other_Sentiment
1,1,204542,Post,['146188473'],https://forums.hardwarezone.com.sg/threads/hou...,NaN,6870671.0,Forums,"Household incomes rose in 2022, income inequal...",singapore - median household income grew in 20...,2023-02-09 10:21:12,income inequality,True,False
2,2,474880,Post,['115313181'],https://forums.hardwarezone.com.sg/threads/bac...,NaN,NaN,Forums,NaN,"hi, want to get views from you all. what does ...",2018-07-05 15:55:32,upper class,True,False
10,10,664200,Post,['113150442'],https://forums.hardwarezone.com.sg/threads/cpf...,NaN,5787403.0,Forums,CPF questions : Medisave after 55 & RA Topup,1. medisave after 55 as i understand excess in...,2018-03-06 23:26:17,medisave,True,False
16,16,815074,Post,['145992640'],https://forums.hardwarezone.com.sg/threads/can...,NaN,6866152.0,Forums,"Can CDAC directors OYK, Sun XL, Koh PK, Chee H...",cdac audited report fy 2021 read cdac audited ...,2023-01-27 00:43:39,cdac,True,False
24,24,789529,Post,['146795555'],https://forums.hardwarezone.com.sg/threads/new...,NaN,NaN,Forums,NaN,accident victim can claim govt medical subsidi...,2023-03-22 11:59:32,medical expenses,True,False


In [14]:
result.to_csv("hardware_zone_clean_data_with_detail.csv", index=False)